In [1]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
import os

In [2]:
MODEL_PATH = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Pruning/mobilenetv3Large_trashnet_pruned_20-30-40"

In [3]:
try:
    # Jika model SUDAH strip_pruning
    model = tf.keras.models.load_model(MODEL_PATH)
    print("Model berhasil dimuat (tanpa pruning wrapper).")
except:
    # Jika model BELUM strip_pruning
    model = tf.keras.models.load_model(
        MODEL_PATH,
        custom_objects={
            "PruneLowMagnitude": tfmot.sparsity.keras.PruneLowMagnitude
        }
    )
    print("Model berhasil dimuat (dengan pruning wrapper).")

model.summary()





Model berhasil dimuat (tanpa pruning wrapper).
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3large (Function  (None, 7, 7, 960)         2996352   
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 960)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 64)                61504     
                                                                 
 dense_1 (Dense)             (None, 6)                 390       
                                                                 
Total params: 3058246 (11.67 MB)
Trainable params: 61894 (241.77 KB)
Non-trainable params: 2996352 (11.43 MB)
___________________________

In [5]:
# 4. CEK LAYER YANG TERDAPAT PRUNING WRAPPER
# (berguna jika model belum di-strip)
print("\nLayer yang menggunakan pruning wrapper")
for layer in model.layers:
    if "prune_low_magnitude" in layer.name.lower():
        print(f"Pruned Layer: {layer.name}")


Layer yang menggunakan pruning wrapper


In [6]:
# 5. CEK SPARSITY (BOBOT NOL) PER LAYER
print("\nPersentase bobot nol per layer")
for layer in model.layers:
    weights = layer.get_weights()
    if weights:
        total_weights = sum(w.size for w in weights)
        zero_weights = sum(np.sum(w == 0) for w in weights)
        sparsity = (zero_weights / total_weights) * 100
        print(f"{layer.name}: {sparsity:.2f}% bobot nol")


Persentase bobot nol per layer
MobilenetV3large: 0.00% bobot nol
dense: 0.00% bobot nol
dense_1: 0.00% bobot nol


In [8]:
# 6. CEK FILTER TERPANGKAS (STRUCTURED PRUNING)
# Khusus Conv2D / DepthwiseConv2D
print("\nFilter Conv2D yang terpangkas")
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D) or \
       isinstance(layer, tf.keras.layers.DepthwiseConv2D):

        weights = layer.get_weights()
        if weights:
            kernel = weights[0]

            # Conv2D: (H, W, Cin, Cout)
            # DepthwiseConv2D: (H, W, Cin, depth_multiplier)
            filter_norm = np.sum(np.abs(kernel), axis=(0,1,2))
            pruned_filters = np.sum(filter_norm == 0)
            total_filters = filter_norm.shape[0]

            print(f"{layer.name}: {pruned_filters}/{total_filters} filter terpangkas")


Filter Conv2D yang terpangkas


In [9]:
# 7. CEK NEURON TERPANGKAS (DENSE LAYER)
print("\nNeuron Dense yang terpangkas")
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Dense):
        weights = layer.get_weights()
        if weights:
            kernel = weights[0]  # (input, units)
            neuron_norm = np.sum(np.abs(kernel), axis=0)
            pruned_neurons = np.sum(neuron_norm == 0)
            total_neurons = kernel.shape[1]

            print(f"{layer.name}: {pruned_neurons}/{total_neurons} neuron terpangkas")


Neuron Dense yang terpangkas
dense: 0/64 neuron terpangkas
dense_1: 0/6 neuron terpangkas
